In [ ]:
pip install pandas

In [ ]:
pip install prophet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask, request
import threading
import time
from prophet import Prophet


In [ ]:

app = Flask(__name__)

@app.route('/run', methods=['POST'])
def run_code():
    # Carregar a base de dados
    data = pd.read_csv("large_phishing_data_with_email.csv")

    # Preprocessamento: Convertendo 'Caiu no Phishing' e 'Reportou' para valores binários
    data['Caiu no Phishing'] = data['Caiu no Phishing'].apply(lambda x: 1 if x == 'Sim' else 0)
    data['Reportou'] = data['Reportou'].apply(lambda x: 1 if x == 'Sim' else 0)

    # Agrupando dados para contar incidentes de phishing e reportes por empresa
    phishing_counts = data.groupby('Nome da Empresa')['Caiu no Phishing'].sum().reset_index()
    report_counts = data.groupby('Nome da Empresa')['Reportou'].sum().reset_index()

    # Configuração do estilo dos gráficos
    sns.set(style="whitegrid")

    # Gráfico de Incidentes de Phishing por Empresa
    plt.figure(figsize=(12, 6))
    sns.barplot(x="Caiu no Phishing", y="Nome da Empresa", data=phishing_counts.sort_values(by="Caiu no Phishing", ascending=False))
    plt.title("Número de Incidentes de Phishing por Empresa")
    plt.xlabel("Número de Incidentes de Phishing")
    plt.ylabel("Nome da Empresa")
    plt.show()

    # Gráfico de Reportes de Phishing por Empresa
    plt.figure(figsize=(12, 6))
    sns.barplot(x="Reportou", y="Nome da Empresa", data=report_counts.sort_values(by="Reportou", ascending=False))
    plt.title("Número de Reportes de Phishing por Empresa")
    plt.xlabel("Número de Reportes de Phishing")
    plt.ylabel("Nome da Empresa")
    plt.show()

# Configurar o servidor Flask em uma thread separada
def run_app():
    app.run(port=8000)

thread = threading.Thread(target=run_app)
thread.start()


In [ ]:
app = Flask(__name__)

@app.route('/run', methods=['POST'])
def run_code():
    # Carregar a base de dados
    data = pd.read_csv("large_phishing_data_with_email.csv")

    # Pré-processamento: Filtrando apenas os incidentes de phishing
    data['Data'] = pd.to_datetime(data['Data'])  # Garantindo que a coluna "Data" seja datetime
    phishing_data = data[data['Caiu no Phishing'] == "Sim"]

    # Contagem de incidentes de phishing por data
    daily_phishing = phishing_data.groupby('Data').size().reset_index(name='Incidentes')

    # Renomeando as colunas para o Prophet
    daily_phishing = daily_phishing.rename(columns={'Data': 'ds', 'Incidentes': 'y'})

    # Instanciando e treinando o modelo Prophet
    model = Prophet(yearly_seasonality=True, daily_seasonality=False)
    model.fit(daily_phishing)

    # Projeção para os próximos anos (ex.: 3 anos)
    future = model.make_future_dataframe(periods=3 * 365)  # 3 anos em dias
    forecast = model.predict(future)

    # Visualização das previsões
    plt.figure(figsize=(10, 6))
    model.plot(forecast)
    plt.title("Projeção de Incidentes de Phishing para os Próximos Anos")
    plt.xlabel("Data")
    plt.ylabel("Número de Incidentes de Phishing")
    plt.show()

    # Visualização de componentes da previsão (tendência e sazonalidade)
    plt.figure(figsize=(10, 6))
    model.plot_components(forecast)
    plt.show()

# Configurar o servidor Flask em uma thread separada
def run_app():
    app.run(port=8000)

thread = threading.Thread(target=run_app)
thread.start()
